# 이지어드민

- 수집을 최근 30일로
- 저장은 나중에

In [ ]:
import requests
import psycopg2
from datetime import datetime, timedelta
import time
import pandas as pd
import json
import psycopg2.extras

In [ ]:
def get_ez_order(start_date, end_date, page):


    partner_key = '73c6ac2e3366905793ab35d5a222d572'
    domain_key = '73c6ac2e33669057718abd505d76aa87366d3eed6643997457a0f9a03797aa9f'
   
    # limit = '100'
    

    url = f'http://api2.cloud.ezadmin.co.kr/ezadmin/function.php?partner_key={partner_key}&domain_key={domain_key}&action=get_order_info&date_type=collect_date&start_date={start_date}&end_date={end_date}&limit=1000&page={page}'
    res = requests.get(url)

    if res.status_code == 200:
        return res.json()['data']
    else:
        return ValueError

In [ ]:
def save_ez_order(cursor, data):

    columns = ['"seq"', '"shop_id"', '"shop_name"', '"order_id"', '"order_id_seq"', '"shop_product_id"', '"product_name"', '"options"', '"qty"', '"order_name"', '"order_mobile"', '"order_tel"', '"recv_name"', '"recv_mobile"', '"recv_tel"', '"recv_address"', '"recv_zip"', '"memo"', '"status"', '"order_cs"', '"collect_date"', '"order_date"', '"trans_date"', '"trans_date_pos"', '"supply_price"', '"amount"', '"extra_money"', '"trans_corp"', '"trans_no"', '"trans_who"', '"prepay_price"', '"gift"', '"hold"', '"org_seq"', '"deal_no"', '"cust_id"', '"order_products"']
    rows = [(d.get('seq'), d.get('shop_id'), d.get('shop_name'), d.get('order_id'), d.get('order_id_seq'), d.get('shop_product_id'), d.get('product_name'), d.get('options'), d.get('qty'), d.get('order_name'), d.get('order_mobile'), d.get('order_tel'), d.get('recv_name'), d.get('recv_mobile'), d.get('recv_tel'), d.get('recv_address'), d.get('recv_zip'), d.get('memo'), d.get('status'), d.get('order_cs'), d.get('collect_date'), d.get('order_date'), d.get('trans_date'), d.get('trans_date_pos'), d.get('supply_price'), d.get('amount'), d.get('extra_money'), d.get('trans_corp'), d.get('trans_no'), d.get('trans_who'), d.get('prepay_price'), d.get('gift'), d.get('hold'), d.get('org_seq'), d.get('deal_no'), d.get('cust_id'), json.dumps(d.get('order_products'))) for d in data]
    
    query = """
    INSERT INTO ez_order ({}) VALUES %s
    ON CONFLICT ("seq") DO UPDATE
    SET {}
    ;
""".format(', '.join(columns), ', '.join('{} = EXCLUDED.{}'.format(column, column) for column in columns))
    
    psycopg2.extras.execute_values(cursor, query, rows)

In [ ]:
start_date = datetime.strptime("2023-04-30", "%Y-%m-%d")
end_date = datetime.strptime("2023-06-01", "%Y-%m-%d")
# start_date = datetime.today() - timedelta(days=30)
# end_date = datetime.today()

date_list = []
current_date = start_date

while current_date <= end_date:
    formatted_date = current_date.strftime("%Y-%m-%d")
    date_list.append(formatted_date)
    current_date += timedelta(days=1)

In [ ]:
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()
data_list = []

for date in date_list:
    page = 1
    data = ['init']
    while len(data) > 0:
        data = get_ez_order(date, date, str(page))
        data_list += data
        print(len(data), date, page)
        print(data)
        page += 1


save_ez_order(cursor, data_list) 


conn.commit()
conn.close()

In [ ]:
data = get_ez_order('2023-07-14', '2023-07-14', '1')

# 네이버

- 테스트로 테이블을 하나 만들어서 데이터 검증 필요
    - 2023 한달 단위로 비교
- 최근 30일로 만들기

In [ ]:
import requests
import bcrypt
import pybase64
import time
import urllib.request
import urllib.parse
from datetime import datetime, timedelta
import pandas as pd
import psycopg2.extras
import http.client
import json


now = datetime.now()
date_list = []
for i in range(50, 100):
    date = now - timedelta(days=i)
    date_list.append(date.astimezone().isoformat())


date_list_settlement = []
for i in range(50, 100):
    date = now - timedelta(days=i)
    date_list_settlement.append(date.strftime("%Y-%m-%d"))


settleType_list = [
    'NORMAL_SETTLE_ORIGINAL',
    'NORMAL_SETTLE_AFTER_CANCEL',
    'NORMAL_SETTLE_BEFORE_CANCEL',
    'QUICK_SETTLE_ORIGINAL',
    'QUICK_SETTLE_CANCEL',
    'QUANTITY_CANCEL_DEDUCTION',
    'QUANTITY_CANCEL_RESTORE' 
]


def get_token(client_id, client_secret, type_="SELF") -> str:
    timestamp = str(int((time.time()-3) * 1000))
    pwd = f'{client_id}_{timestamp}'
    hashed = bcrypt.hashpw(pwd.encode('utf-8'), client_secret.encode('utf-8'))
    client_secret_sign = pybase64.standard_b64encode(hashed).decode('utf-8')

    headers = {"content-type": "application/x-www-form-urlencoded"}
    data_ = {
        "client_id": client_id,
        "timestamp": timestamp,
        "client_secret_sign": client_secret_sign,
        "grant_type": "client_credentials",
        "type": type_
    }

    query = urllib.parse.urlencode(data_)
    url = 'https://api.commerce.naver.com/external/v1/oauth2/token?' + query
    res = requests.post(url=url, headers=headers)
    res_data = res.json()

    while True:
        if 'access_token' in res_data:
            token = res_data['access_token']
            return token
        else:
            print(f'[{res_data}] 토큰 요청 실패')
            time.sleep(1)


def get_naver_order(token, lastChangedFrom, moreSequence):
    
    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/last-changed-statuses'
   
    params = {
            'lastChangedFrom' : lastChangedFrom, #조회시작일시
            'moreSequence' : moreSequence,
        }

    res = requests.get(url=url, headers=headers, params=params)
    res_data = res.json()

    if 'data' not in res_data: #조회된 정보가 없을 경우 data키 없음
        print('주문 내역 없음')
        return False

    return res_data


def save_naver_order(cursor, data):
    query = """
        INSERT INTO naver_order ("lastChangedType", "orderId", "productOrderId", "productOrderStatus", "paymentDate", "lastChangedDate", "receiverAddressChanged")
        VALUES %s
        ON CONFLICT ("productOrderId") DO UPDATE
        SET "lastChangedType" = EXCLUDED."lastChangedType", 
        "orderId" = EXCLUDED."orderId", 
        "productOrderStatus" = EXCLUDED."productOrderStatus", 
        "paymentDate" = EXCLUDED."paymentDate", 
        "lastChangedDate" = EXCLUDED."lastChangedDate", 
        "receiverAddressChanged" = EXCLUDED."receiverAddressChanged";
    """

    rows = [(d['lastChangedType'], d['orderId'], d['productOrderId'], d['productOrderStatus'], d.get('paymentDate'), d['lastChangedDate'], d['receiverAddressChanged']) for d in data]

    # execute_values를 사용하여 여러 행 삽입
    psycopg2.extras.execute_values(cursor, query, rows)


def get_naver_order_detail(token, productOrderId):

    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/query'
    
    params = {
        'productOrderIds' : productOrderId
    }

    res = requests.post(url=url, headers=headers, json=params)
    res_data = res.json()
    
    if 'data' not in res_data:
        return False
    
    return res_data['data']


def save_naver_order_detail(cursor, data):

    columns = ['"cancelApprovalDate"', '"cancelCompletedDate"', '"cancelDetailedReason"', '"cancelReason"', '"claimRequestDate"', '"claimStatus"', '"refundExpectedDate"', '"refundStandbyReason"', '"refundStandbyStatus"', '"requestChannel"', '"deliveredDate"', '"deliveryCompany"', '"deliveryMethod"', '"deliveryStatus"', '"isWrongTrackingNumber"', '"pickupDate"', '"sendDate"', '"trackingNumber"', '"wrongTrackingNumberRegisteredDate"', '"wrongTrackingNumberType"', '"claimDeliveryFeeDemandAmount"', '"claimDeliveryFeePayMeans"', '"claimDeliveryFeePayMethod"', '"collectAddress"', '"collectCompletedDate"', '"collectDeliveryCompany"', '"collectDeliveryMethod"', '"collectStatus"', '"collectTrackingNumber"', '"etcFeeDemandAmount"', '"etcFeePayMeans"', '"etcFeePayMethod"', '"exchangeDetailedReason"', '"exchangeReason"', '"holdbackDetailedReason"', '"holdbackReason"', '"holdbackStatus"', '"reDeliveryMethod"', '"reDeliveryStatus"', '"reDeliveryCompany"', '"chargeAmountPaymentAmount"', '"checkoutAccumulationPaymentAmount"', '"generalPaymentAmount"', '"naverMileagePaymentAmount"', '"orderDate"', '"orderDiscountAmount"', '"orderId"', '"ordererId"', '"ordererName"', '"ordererTel"', '"paymentDate"', '"paymentDueDate"', '"paymentMeans"', '"isDeliveryMemoParticularInput"', '"payLocationType"', '"ordererNo"', '"payLaterPaymentAmount"', '"claimType"', '"decisionDate"', '"delayedDispatchDetailedReason"', '"delayedDispatchReason"', '"deliveryDiscountAmount"', '"deliveryFeeAmount"', '"deliveryPolicyType"', '"expectedDeliveryMethod"', '"freeGift"', '"mallId"', '"optionCode"', '"optionPrice"', '"packageNumber"', '"placeOrderDate"', '"placeOrderStatus"', '"productClass"', '"productDiscountAmount"', '"productId"', '"productName"', '"productOption"', '"productOrderId"', '"productOrderStatus"', '"quantity"', '"sectionDeliveryFee"', '"sellerProductCode"', '"shippingDueDate"', '"shippingFeeType"', '"shippingAddress"', '"shippingMemo"', '"totalPaymentAmount"', '"totalProductAmount"', '"unitPrice"', '"sellerBurdenDiscountAmount"', '"commissionRatingType"', '"commissionPrePayStatus"', '"paymentCommission"', '"saleCommission"', '"expectedSettlementAmount"', '"inflowPath"', '"itemNo"', '"optionManageCode"', '"sellerCustomCode1"', '"sellerCustomCode2"', '"claimId"', '"channelCommission"', '"individualCustomUniqueCode"', '"productImediateDiscountAmount"', '"productProductDiscountAmount"', '"productMultiplePurchaseDiscountAmount"', '"sellerBurdenImediateDiscountAmount"', '"sellerBurdenProductDiscountAmount"', '"sellerBurdenMultiplePurchaseDiscountAmount"', '"knowledgeShoppingSellingInterlockCommission"', '"giftReceivingStatus"', '"sellerBurdenStoreDiscountAmount"', '"sellerBurdenMultiplePurchaseDiscountType"', '"logisticsCompanyId"', '"logisticsCenterId"', '"arrivalGuaranteeDate"', '"deliveryAttributeType"', '"returnDetailedReason"', '"returnReason"', '"returnReceiveAddress"', '"returnCompletedDate"', '"holdbackConfigDate"', '"holdbackConfigurer"', '"holdbackReleaseDate"', '"holdbackReleaser"', '"claimDeliveryFeeProductOrderIds"', '"claimDeliveryFeeDiscountAmount"', '"remoteAreaCostChargeAmount"']
    rows = [(d.get('cancelApprovalDate'), d.get('cancelCompletedDate'), d.get('cancelDetailedReason'), d.get('cancelReason'), d.get('claimRequestDate'), d.get('claimStatus'), d.get('refundExpectedDate'), d.get('refundStandbyReason'), d.get('refundStandbyStatus'), d.get('requestChannel'), d.get('deliveredDate'), d.get('deliveryCompany'), d.get('deliveryMethod'), d.get('deliveryStatus'), d.get('isWrongTrackingNumber'), d.get('pickupDate'), d.get('sendDate'), d.get('trackingNumber'), d.get('wrongTrackingNumberRegisteredDate'), d.get('wrongTrackingNumberType'), d.get('claimDeliveryFeeDemandAmount'), d.get('claimDeliveryFeePayMeans'), d.get('claimDeliveryFeePayMethod'), json.dumps(d.get('collectAddress'), ensure_ascii=False), d.get('collectCompletedDate'), d.get('collectDeliveryCompany'), d.get('collectDeliveryMethod'), d.get('collectStatus'), d.get('collectTrackingNumber'), d.get('etcFeeDemandAmount'), d.get('etcFeePayMeans'), d.get('etcFeePayMethod'), d.get('exchangeDetailedReason'), d.get('exchangeReason'), d.get('holdbackDetailedReason'), d.get('holdbackReason'), d.get('holdbackStatus'), d.get('reDeliveryMethod'), d.get('reDeliveryStatus'), d.get('reDeliveryCompany'), d.get('chargeAmountPaymentAmount'), d.get('checkoutAccumulationPaymentAmount'), d.get('generalPaymentAmount'), d.get('naverMileagePaymentAmount'), d.get('orderDate'), d.get('orderDiscountAmount'), d['orderId'], d.get('ordererId'), d.get('ordererName'), d.get('ordererTel'), d.get('paymentDate'), d.get('paymentDueDate'), d.get('paymentMeans'), d.get('isDeliveryMemoParticularInput'), d.get('payLocationType'), d.get('ordererNo'), d.get('payLaterPaymentAmount'), d.get('claimType'), d.get('decisionDate'), d.get('delayedDispatchDetailedReason'), d.get('delayedDispatchReason'), d.get('deliveryDiscountAmount'), d.get('deliveryFeeAmount'), d.get('deliveryPolicyType'), d.get('expectedDeliveryMethod'), d.get('freeGift'), d.get('mallId'), d.get('optionCode'), d.get('optionPrice'), d.get('packageNumber'), d.get('placeOrderDate'), d.get('placeOrderStatus'), d.get('productClass'), d.get('productDiscountAmount'), d.get('productId'), d.get('productName'), d.get('productOption'), d['productOrderId'], d.get('productOrderStatus'), d.get('quantity'), d.get('sectionDeliveryFee'), d.get('sellerProductCode'), d.get('shippingDueDate'), d.get('shippingFeeType'), json.dumps(d.get('shippingAddress'), ensure_ascii=False), d.get('shippingMemo'), d.get('totalPaymentAmount'), d.get('totalProductAmount'), d.get('unitPrice'), d.get('sellerBurdenDiscountAmount'), d.get('commissionRatingType'), d.get('commissionPrePayStatus'), d.get('paymentCommission'), d.get('saleCommission'), d.get('expectedSettlementAmount'), d.get('inflowPath'), d.get('itemNo'), d.get('optionManageCode'), d.get('sellerCustomCode1'), d.get('sellerCustomCode2'), d.get('claimId'), d.get('channelCommission'), d.get('individualCustomUniqueCode'), d.get('productImediateDiscountAmount'), d.get('productProductDiscountAmount'), d.get('productMultiplePurchaseDiscountAmount'), d.get('sellerBurdenImediateDiscountAmount'), d.get('sellerBurdenProductDiscountAmount'), d.get('sellerBurdenMultiplePurchaseDiscountAmount'), d.get('knowledgeShoppingSellingInterlockCommission'), d.get('giftReceivingStatus'), d.get('sellerBurdenStoreDiscountAmount'), d.get('sellerBurdenMultiplePurchaseDiscountType'), d.get('logisticsCompanyId'), d.get('logisticsCenterId'), d.get('arrivalGuaranteeDate'), d.get('deliveryAttributeType'), d.get('returnDetailedReason'), d.get('returnReason'), json.dumps(d.get('returnReceiveAddress'), ensure_ascii=False), d.get('returnCompletedDate'), d.get('holdbackConfigDate'), d.get('holdbackConfigurer'), d.get('holdbackReleaseDate'), d.get('holdbackReleaser'), d.get('claimDeliveryFeeProductOrderIds'), d.get('claimDeliveryFeeDiscountAmount'), d.get('remoteAreaCostChargeAmount')) for d in data]
    
    query = """
    INSERT INTO naver_order_product2 ({}) VALUES %s
    ON CONFLICT ("productOrderId") DO UPDATE
    SET {}
    ;
""".format(', '.join(columns), ', '.join('{} = EXCLUDED.{}'.format(column, column) for column in columns))
    
    psycopg2.extras.execute_values(cursor, query, rows)


def get_naver_settlement_case(token, searchDate, periodType, settleType, pageNumber, pageSize):

    conn = http.client.HTTPSConnection("api.commerce.naver.com")
    token = get_token(client_id='6rmQh0T5jWd9KuJeHIlzU0', client_secret='$2a$04$KJqyFL0J3K5XJdxuTkKZFu')
    headers = { 'Authorization': f"Bearer {token}" }

    conn.request("GET", f"/external/v1/pay-settle/settle/case?searchDate={searchDate}&periodType={periodType}&settleType={settleType}&pageNumber={pageNumber}&pageSize={pageSize}", headers=headers)

    res = conn.getresponse()
    data = res.read()

    return json.loads(data.decode("utf-8"))


def save_naver_settlement_case(cursor, data):

    columns = ['"settleBasisDate"', '"settleExpectDate"', '"settleCompleteDate"', '"orderId"', '"productOrderId"', '"productOrderType"', '"settleType"', '"productId"', '"productName"', '"purchaserName"', '"paySettleAmount"', '"totalPayCommissionAmount"', '"freeInstallmentCommissionAmount"', '"sellingInterlockCommissionAmount"', '"benefitSettleAmount"', '"settleExpectAmount"', '"merchantId"', '"merchantName"', '"contractNo"']
    rows = [list(d.values()) for d in data]
    
    query = """
    INSERT INTO naver_settlement_case ({}) VALUES %s
    ON CONFLICT ("productOrderId", "productOrderType", "settleType") DO UPDATE
    SET {}
    ;
""".format(', '.join(columns), ', '.join('{} = EXCLUDED.{}'.format(column, column) for column in columns))
    
    psycopg2.extras.execute_values(cursor, query, rows)


def get_naver_settlement_daily(token, startDate, endDate, pageNumber, pageSize):

    conn = http.client.HTTPSConnection("api.commerce.naver.com")
    token = get_token(client_id='6rmQh0T5jWd9KuJeHIlzU0', client_secret='$2a$04$KJqyFL0J3K5XJdxuTkKZFu')
    headers = { 'Authorization': f"Bearer {token}" }

    conn.request("GET", f"/external/v1/pay-settle/settle/daily?startDate={startDate}&endDate={endDate}&pageNumber={pageNumber}&pageSize={pageSize}", headers=headers)


    res = conn.getresponse()
    data = res.read()

    return json.loads(data.decode("utf-8"))


def save_naver_settlement_daily(cursor, data):

    columns = ['"settleBasisStartDate"', '"settleBasisEndDate"', '"settleExpectDate"', '"settleCompleteDate"', '"settleAmount"', '"paySettleAmount"', '"commissionSettleAmount"', '"benefitSettleAmount"', '"deductionRestoreSettleAmount"', '"payHoldbackAmount"', '"minusChargeAmount"', '"differenceSettleAmount"', '"returnCareSettleAmount"', '"quickSettleAmount"', '"settleMethodType"', '"bankType"', '"depositorName"', '"accountNo"', '"merchantId"', '"merchantName"']
    rows = [list(d.values()) for d in data]
    
    query = """
    INSERT INTO naver_settlement_daily ({}) VALUES %s
    ON CONFLICT ("settleBasisStartDate") DO UPDATE
    SET {}
    ;
""".format(', '.join(columns), ', '.join('{} = EXCLUDED.{}'.format(column, column) for column in columns))
    
    psycopg2.extras.execute_values(cursor, query, rows)



In [ ]:
# 건별 정산내역
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()
data_list = []


for date in date_list_settlement:
    for settleType in settleType_list:
        token = get_token(client_id='6rmQh0T5jWd9KuJeHIlzU0', client_secret='$2a$04$KJqyFL0J3K5XJdxuTkKZFu')
        data = get_naver_settlement_case(token, date, 'SETTLE_CASEBYCASE_SETTLE_COMPLETE_DATE', settleType, '1', '1000')
        print(data)
        data_list += data['elements']
        time.sleep(0.5)


save_naver_settlement_case(cursor, data_list)


conn.commit()
conn.close()

In [ ]:
# 일별 정산내역
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()
data_list = []


for date in date_list_settlement:
    token = get_token(client_id='6rmQh0T5jWd9KuJeHIlzU0', client_secret='$2a$04$KJqyFL0J3K5XJdxuTkKZFu')
    data = get_naver_settlement_daily(token, date, date, '1', '1000')
    print(data)
    data_list += data['elements']    
    time.sleep(1)


save_naver_settlement_daily(cursor, data_list)


conn.commit()
conn.close()

In [ ]:
# 상품주문번호
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()
token = get_token(client_id='6rmQh0T5jWd9KuJeHIlzU0', client_secret='$2a$04$KJqyFL0J3K5XJdxuTkKZFu')
# print(f'발급된 토큰 : ', token)


data_list = []


for date in date_list:
    data = get_naver_order(token, date, None)
    data_list += data['data']['lastChangeStatuses']
    print(date)
    time.sleep(1)
    

    while 'more' in data['data'].keys():
        moreFrom = data['data']['more']['moreFrom']
        moreSequence = data['data']['more']['moreSequence']
        data = get_naver_order(token, moreFrom, moreSequence)
        data_list += data['data']['lastChangeStatuses']
        
        time.sleep(1) # 나중엔 5초로 변경


save_naver_order(cursor, data_list)


conn.commit()
conn.close()

In [ ]:
# 주문상세
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()
token = get_token(client_id='6rmQh0T5jWd9KuJeHIlzU0', client_secret='$2a$04$KJqyFL0J3K5XJdxuTkKZFu')
productOrderId = [d['productOrderId'] for d in data_list]
data_list_2 = []


for i in range(0, len(productOrderId), 300):
    data = get_naver_order_detail(token, productOrderId[i:i+300])
    print(data)
    for d in data:
        data_dict = {}
        keys = d.keys()
        for key in keys:
            data_dict.update(d[key])
        data_list_2.append(data_dict)
    time.sleep(1)


save_naver_order_detail(cursor, data_list_2)


conn.commit()
conn.close()

# 쿠팡

In [62]:
import os
import time
import hmac, hashlib
import urllib.parse
import urllib.request
import ssl
import json
import pandas as pd
from datetime import datetime, timedelta
import psycopg2.extras


now = datetime.now()
date_list = []
for i in range(1, 100):
    date = now - timedelta(days=i)
    date_list.append(date.strftime("%Y-%m-%d"))


order_status_list = ['ACCEPT', 'INSTRUCT', 'DEPARTURE', 'DELIVERING', 'FINAL_DELIVERY']
cs_status_list = ['RU', 'UC', 'CC', 'PR']


def get_coupang_api(path, query):

    os.environ['TZ'] = 'GMT+0'
    date=(datetime.now() - timedelta(hours = 9)).strftime('%y%m%dT%H%M%SZ')
    method = "GET"
    print(date)

    message = date+method+path+query
    accesskey = "73814036-d7f4-4d56-a254-35908249a0c6"
    secretkey = "86ecbb8dcd4f4d9c4bd83f724ed4bfb4919ba0cc"

    #********************************************************#
    #authorize, demonstrate how to generate hmac signature here
    signature=hmac.new(secretkey.encode('utf-8'),message.encode('utf-8'),hashlib.sha256).hexdigest()
    authorization  = "CEA algorithm=HmacSHA256, access-key="+accesskey+", signed-date="+date+", signature="+signature
    #print out the hmac key
    #print(authorization)
    #********************************************************#

    # ************* SEND THE REQUEST *************
    url = "https://api-gateway.coupang.com"+path+"?%s" % query

    req = urllib.request.Request(url)

    req.add_header("Content-type","application/json;charset=UTF-8")
    req.add_header("Authorization",authorization)

    req.get_method = lambda: method

    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    try:
        resp = urllib.request.urlopen(req,context=ctx)
    except urllib.request.HTTPError as e:
        print(e.code)
        print(e.reason)
        print(e.fp.read())
    except urllib.request.URLError as e:
        print(e.errno)
        print(e.reason)
        print(e.fp.read())
    else:
        # 200
        # body = resp.read().decode(resp.headers.get_content_charset())
        body = json.loads(resp.read())
        return body


# 쿠팡 주문
def get_coupang_order(createdAtFrom, createdAtTo, status, nextToken):

    path = f"/v2/providers/openapi/apis/api/v4/vendors/A00197911/ordersheets"
    if nextToken == None:
        query = urllib.parse.urlencode({"createdAtFrom": createdAtFrom, "createdAtTo": createdAtTo, "status": status})
    else:
        query = urllib.parse.urlencode({"createdAtFrom": createdAtFrom, "createdAtTo": createdAtTo, "status": status, "nextToken" : nextToken})


    return get_coupang_api(path, query)



def save_coupang_order(cursor, data):

    columns = ['"shipmentBoxId"', '"orderId"', '"orderedAt"', '"orderer"', '"paidAt"', '"status"', '"shippingPrice"', '"remotePrice"', '"remoteArea"', '"parcelPrintMessage"', '"splitShipping"', '"ableSplitShipping"', '"receiver"', '"orderItems"', '"overseaShippingInfoDto"', '"deliveryCompanyName"', '"invoiceNumber"', '"refer"', '"shipmentType"', '"lastChangedType"', '"inTrasitDateTime"', '"deliveredDate"']
    rows = [(d.get('shipmentBoxId'), d['orderId'], d.get('orderedAt'), json.dumps(d.get('orderer'), ensure_ascii=False), d.get('paidAt'), d.get('status'), d.get('shippingPrice'), d.get('remotePrice'), d.get('remoteArea'), d.get('parcelPrintMessage'), d.get('splitShipping'), d.get('ableSplitShipping'), json.dumps(d.get('receiver'), ensure_ascii=False), json.dumps(d.get('orderItems'), ensure_ascii=False), json.dumps(d.get('overseaShippingInfoDto'), ensure_ascii=False), d.get('deliveryCompanyName'), d.get('invoiceNumber'), d.get('refer'), d.get('shipmentType'), d.get('lastChangedType'), d.get('inTrasitDateTime'), d.get('deliveredDate')) for d in data]
    
    query = """
    INSERT INTO coupang_order ({}) VALUES %s
    ON CONFLICT ("orderId") DO UPDATE
    SET {}
    ;
""".format(', '.join(columns), ', '.join('{} = EXCLUDED.{}'.format(column, column) for column in columns))
    
    psycopg2.extras.execute_values(cursor, query, rows)


def get_coupang_return_cancel(createdAtFrom, createdAtTo, status, nextToken):
    
    path = f"/v2/providers/openapi/apis/api/v4/vendors/A00197911/returnRequests"
    
    if status == None:
        if nextToken == None:
            query = urllib.parse.urlencode({"createdAtFrom": createdAtFrom, "createdAtTo": createdAtTo, "cancelType": 'CANCEL'})
        else:
            query = urllib.parse.urlencode({"createdAtFrom": createdAtFrom, "createdAtTo": createdAtTo, "cancelType": 'CANCEL', "nextToken" : nextToken})
    else:
        if nextToken == None:
            query = urllib.parse.urlencode({"createdAtFrom": createdAtFrom, "createdAtTo": createdAtTo, "status": status})
        else:
            query = urllib.parse.urlencode({"createdAtFrom": createdAtFrom, "createdAtTo": createdAtTo, "status": status, "nextToken" : nextToken})

    return get_coupang_api(path, query)


def save_coupang_return_cancel(cursor, data):

    columns = ['"receiptId"', '"orderId"', '"paymentId"', '"receiptType"', '"receiptStatus"', '"createdAt"', '"modifiedAt"', '"requesterName"', '"requesterPhoneNumber"', '"requesterRealPhoneNumber"', '"requesterAddress"', '"requesterAddressDetail"', '"requesterZipCode"', '"cancelReasonCategory1"', '"cancelReasonCategory2"', '"cancelReason"', '"cancelCountSum"', '"returnDeliveryId"', '"returnDeliveryType"', '"releaseStopStatus"', '"enclosePrice"', '"faultByType"', '"preRefund"', '"completeConfirmType"', '"completeConfirmDate"', '"returnItems"', '"returnDeliveryDtos"', '"reasonCode"', '"reasonCodeText"', '"returnShippingCharge"']
    rows = [(d.get('receiptId'), d.get('orderId'), d.get('paymentId'), d.get('receiptType'), d.get('receiptStatus'), d.get('createdAt'), d.get('modifiedAt'), d.get('requesterName'), d.get('requesterPhoneNumber'), d.get('requesterRealPhoneNumber'), d.get('requesterAddress'), d.get('requesterAddressDetail'), d.get('requesterZipCode'), d.get('cancelReasonCategory1'), d.get('cancelReasonCategory2'), d.get('cancelReason'), d.get('cancelCountSum'), d.get('returnDeliveryId'), d.get('returnDeliveryType'), d.get('releaseStopStatus'), d.get('enclosePrice'), d.get('faultByType'), d.get('preRefund'), d.get('completeConfirmType'), d.get('completeConfirmDate'), json.dumps(d.get('returnItems'), ensure_ascii=False), json.dumps(d.get('returnDeliveryDtos'), ensure_ascii=False), d.get('reasonCode'), d.get('reasonCodeText'), d.get('returnShippingCharge')) for d in data]
    
    query = """
    INSERT INTO coupang_return_cancel ({}) VALUES %s
    ON CONFLICT ("receiptId") DO UPDATE
    SET {}
    ;
""".format(', '.join(columns), ', '.join('{} = EXCLUDED.{}'.format(column, column) for column in columns))
    
    psycopg2.extras.execute_values(cursor, query, rows)



# 쿠팡 정산
def get_coupang_settlement_case(recognitionDateFrom, recognitionDateTo, token):
    
    path = "/v2/providers/openapi/apis/api/v1/revenue-history"
    query = urllib.parse.urlencode({"vendorId" : "A00197911", "recognitionDateFrom": recognitionDateFrom, "recognitionDateTo": recognitionDateTo, "token": token})

    return get_coupang_api(path, query)


def save_coupang_settlement_case(cursor, data):

    columns = ['"orderId"', '"saleType"', '"saleDate"', '"recognitionDate"', '"settlementDate"', '"finalSettlementDate"', '"deliveryFee"', '"taxType"', '"productId"', '"productName"', '"vendorItemId"', '"vendorItemName"', '"salePrice"', '"quantity"', '"coupangDiscountCoupon"', '"discountCouponPolicyAgreement"', '"saleAmount"', '"sellerDiscountCoupon"', '"downloadableCoupon"', '"serviceFee"', '"serviceFeeVat"', '"serviceFeeRatio"', '"settlementAmount"', '"couranteeFeeRatio"', '"couranteeFee"', '"couranteeFeeVat"', '"storeFeeDiscountVat"', '"storeFeeDiscount"', '"externalSellerSkuCode"']
    rows = [(d.get('orderId'), d.get('saleType'), d.get('saleDate'), d.get('recognitionDate'), d.get('settlementDate'), d.get('finalSettlementDate'), json.dumps(d.get('deliveryFee'), ensure_ascii=False), d['items'][0]['taxType'], d['items'][0]['productId'], d['items'][0]['productName'], d['items'][0]['vendorItemId'], d['items'][0]['vendorItemName'], d['items'][0]['salePrice'], d['items'][0]['quantity'], d['items'][0]['coupangDiscountCoupon'], d['items'][0]['discountCouponPolicyAgreement'], d['items'][0]['saleAmount'], d['items'][0]['sellerDiscountCoupon'], d['items'][0]['downloadableCoupon'], d['items'][0]['serviceFee'], d['items'][0]['serviceFeeVat'], d['items'][0]['serviceFeeRatio'], d['items'][0]['settlementAmount'], d['items'][0]['couranteeFeeRatio'], d['items'][0]['couranteeFee'], d['items'][0]['couranteeFeeVat'], d['items'][0]['storeFeeDiscountVat'], d['items'][0]['storeFeeDiscount'], d['items'][0]['externalSellerSkuCode']) for d in data]
    
    query = """
    INSERT INTO coupang_settlement_case ({}) VALUES %s
    ON CONFLICT ("orderId", "saleType", "vendorItemId") DO UPDATE
    SET {}
    ;
""".format(', '.join(columns), ', '.join('{} = EXCLUDED.{}'.format(column, column) for column in columns))
    
    psycopg2.extras.execute_values(cursor, query, rows)



def get_coupang_settlement_history(revenueRecognitionYearMonth):
    
    path = "/v2/providers/marketplace_openapi/apis/api/v1/settlement-histories"
    query = urllib.parse.urlencode({"revenueRecognitionYearMonth" : revenueRecognitionYearMonth})

    return get_coupang_api(path, query)    
    

def save_coupang_settlement_history(cursor, data):

    columns = ['"settlementType"', '"settlementDate"', '"revenueRecognitionYearMonth"', '"revenueRecognitionDateFrom"', '"revenueRecognitionDateTo"', '"totalSale"', '"serviceFee"', '"settlementTargetAmount"', '"settlementAmount"', '"lastAmount"', '"pendingReleasedAmount"', '"sellerDiscountCoupon"', '"downloadableCoupon"', '"dedicatedDeliveryAmount"', '"sellerServiceFee"', '"couranteeFee"', '"couranteeCustomerReward"', '"deductionAmount"', '"debtOfLastWeek"', '"finalAmount"', '"bankAccountHolder"', '"bankName"', '"bankAccount"', '"status"', '"storeFeeDiscount"']
    rows = [list(d.values()) for d in data]
    
    query = """
    INSERT INTO coupang_settlement_history ({}) VALUES %s
    ON CONFLICT ("settlementType", "settlementDate", "revenueRecognitionDateFrom", "revenueRecognitionDateTo") DO UPDATE
    SET {}
    ;
""".format(', '.join(columns), ', '.join('{} = EXCLUDED.{}'.format(column, column) for column in columns))
    
    psycopg2.extras.execute_values(cursor, query, rows)

In [ ]:
# 쿠팡 주문 데이터 수집

conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()
data_list = []


for date in date_list:
    for status in order_status_list:
        data = get_coupang_order(date, date, status, None)   
        data_list += data['data']
        print(date, status, data['data'])
        time.sleep(0.5)
        while data['nextToken'] != '':
            data = get_coupang_order(date, date, status, data['nextToken'])
            data_list += data['data']
            time.sleep(0.5)


save_coupang_order(cursor, data_list)


conn.commit()
conn.close()

In [ ]:
# 쿠팡 취소, 반품 데이터 수집
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()


# 취소
data_list = []
for date in date_list:
    data = get_coupang_return_cancel(date, date, None, None)
    data_list += data['data']
    print(date, len(data['data']))
    time.sleep(0.5)
    while data['nextToken'] != '':
        data = get_coupang_return_cancel(date, date, None, data['nextToken'])
        data_list += data['data']
        print(date, len(data['data']))
        time.sleep(0.5)


save_coupang_return_cancel(cursor, data_list)


# 반품
data_list = []
for date in date_list:
    for status in cs_status_list:
        data = get_coupang_return_cancel(date, date, status, None)
        data_list += data['data']
        print(date, len(data['data']))
        time.sleep(0.5)
        while data['nextToken'] != '':
            data = get_coupang_return_cancel(date, date, status, data['nextToken'])
            data_list += data['data']
            print(date, len(data['data']))
            time.sleep(0.5)


save_coupang_return_cancel(cursor, data_list)


conn.commit()
conn.close()

In [63]:
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()
data_list = []


for date in date_list:
    data = get_coupang_settlement_case(date, date, "")
    data_list += data['data']
    print(date, len(data['data']))
    time.sleep(0.5)


save_coupang_settlement_case(cursor, data_list)


conn.commit()
conn.close()

230718T095539Z
2023-07-17 6
230718T095540Z
2023-07-16 0
230718T095541Z
2023-07-15 38
230718T095542Z
2023-07-14 38
230718T095544Z
2023-07-13 50
230718T095546Z
2023-07-12 43
230718T095548Z
2023-07-11 50
230718T095550Z
2023-07-10 3
230718T095551Z
2023-07-09 0
230718T095551Z
2023-07-08 31
230718T095552Z
2023-07-07 49
230718T095554Z
2023-07-06 36
230718T095556Z
2023-07-05 50
230718T095558Z
2023-07-04 50
230718T095559Z
2023-07-03 0
230718T095600Z
2023-07-02 0
230718T095601Z
2023-07-01 31
230718T095602Z
2023-06-30 50
230718T095604Z
2023-06-29 50
230718T095606Z
2023-06-28 49
230718T095608Z
2023-06-27 50
230718T095609Z
2023-06-26 6
230718T095610Z
2023-06-25 0
230718T095611Z
2023-06-24 38
230718T095613Z
2023-06-23 50
230718T095615Z
2023-06-22 49
230718T095617Z
2023-06-21 50
230718T095619Z
2023-06-20 50
230718T095622Z
2023-06-19 10
230718T095623Z
2023-06-18 0
230718T095623Z
2023-06-17 50
230718T095626Z
2023-06-16 50
230718T095628Z
2023-06-15 50
230718T095630Z
2023-06-14 50
230718T095632Z
2023-06-

In [50]:
a = data_list[0]
b = data_list[0]['items'][0]

In [51]:
a.update(b)

In [55]:
data

{'code': 200,
 'message': 'OK',
 'data': [{'orderId': 25100000899805,
   'saleType': 'SALE',
   'saleDate': '2023-07-06',
   'recognitionDate': '2023-07-17',
   'settlementDate': '2023-08-11',
   'finalSettlementDate': '2023-09-01',
   'deliveryFee': {'amount': 3000,
    'fee': 90,
    'feeVat': 9,
    'feeRatio': 3,
    'settlementAmount': 2901,
    'baseAmount': 0,
    'baseFee': 0,
    'baseFeeVat': 0,
    'remoteAmount': 3000,
    'remoteFee': 90,
    'remoteFeeVat': 9},
   'items': [{'taxType': 'TAX',
     'productId': 7319830904,
     'productName': '판토모나 비오틴 하이퍼포머',
     'vendorItemId': 5456714754,
     'vendorItemName': '판토모나 비오틴 하이퍼포머, 240정, 1개',
     'salePrice': 58990,
     'quantity': 1,
     'coupangDiscountCoupon': 0,
     'discountCouponPolicyAgreement': False,
     'saleAmount': 58990,
     'sellerDiscountCoupon': 0,
     'downloadableCoupon': 0,
     'serviceFee': 6253,
     'serviceFeeVat': 625,
     'serviceFeeRatio': 10.6,
     'settlementAmount': 52112,
     'coura

In [52]:
a

{'orderId': 25100000899805,
 'saleType': 'SALE',
 'saleDate': '2023-07-06',
 'recognitionDate': '2023-07-17',
 'settlementDate': '2023-08-11',
 'finalSettlementDate': '2023-09-01',
 'deliveryFee': {'amount': 3000,
  'fee': 90,
  'feeVat': 9,
  'feeRatio': 3,
  'settlementAmount': 2901,
  'baseAmount': 0,
  'baseFee': 0,
  'baseFeeVat': 0,
  'remoteAmount': 3000,
  'remoteFee': 90,
  'remoteFeeVat': 9},
 'items': [{'taxType': 'TAX',
   'productId': 7319830904,
   'productName': '판토모나 비오틴 하이퍼포머',
   'vendorItemId': 5456714754,
   'vendorItemName': '판토모나 비오틴 하이퍼포머, 240정, 1개',
   'salePrice': 58990,
   'quantity': 1,
   'coupangDiscountCoupon': 0,
   'discountCouponPolicyAgreement': False,
   'saleAmount': 58990,
   'sellerDiscountCoupon': 0,
   'downloadableCoupon': 0,
   'serviceFee': 6253,
   'serviceFeeVat': 625,
   'serviceFeeRatio': 10.6,
   'settlementAmount': 52112,
   'couranteeFeeRatio': 0,
   'couranteeFee': 0,
   'couranteeFeeVat': 0,
   'storeFeeDiscountVat': 0,
   'storeFeeDi

In [19]:
import pandas as pd
df = pd.DataFrame(data_list)

In [36]:
df[df.duplicated(subset=['orderId'], keep=False)].to_clipboard()

In [16]:
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()


data = get_coupang_settlement_history('2023-05')
save_coupang_settlement_history(cursor, data)


conn.commit()
conn.close()

230718T065833Z
